# Spring Dynamics: Mass-Spring System
This tutorial demonstrates how to simulate a simple 1D mass-spring system in SOFA. You will learn how to connect two particles with a linear spring, apply constraints, and visualize the resulting oscillation.

### Learning Objectives
- Define a 1D simulation using the `Vec1` template.
- Use `FixedProjectiveConstraint` to anchor a part of the simulation.
- Implement a `SpringForceField` to model elastic interactions.
- Retrieve and plot simulation data using `bokeh`.

---

## 1. Environment Setup
We begin by importing the necessary SOFA modules and initializing the runtime.

In [ ]:
import Sofa
import SofaRuntime
SofaRuntime.init()

## 2. Creating the Scene Graph
We create a `root` node and set gravity to zero. In this tutorial, we focus purely on the spring force without external gravitational influence.

In [ ]:
root = Sofa.Core.Node("root")
root.gravity.value = [0, 0, 0]

## 3. Simulation Components

### Animation Loop and Solver
We use the `DefaultAnimationLoop` and the `EulerExplicitSolver`. The explicit solver is suitable for this simple system, provided the time step and stiffness are well-balanced.

In [ ]:
root.addObject("DefaultAnimationLoop")

# Load the plugin for the Forward (Explicit) ODE Solver
SofaRuntime.importPlugin("Sofa.Component.ODESolver.Forward")
root.addObject("EulerExplicitSolver", name="solver")

### Mechanical State
We use `MechanicalObject` with `template="Vec1"` to represent our particles in a 1D space. We define two particles:
1.  **Particle 0**: At position 0, which will be fixed.
2.  **Particle 1**: At position 1, which will be attached to the spring.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.StateContainer")

initial_position = [[0.0], [1.0]] # 1D positions for two particles
initial_velocity = [[0.0], [0.0]]
root.addObject("MechanicalObject", template="Vec1", name="particles", 
               position=initial_position, velocity=initial_velocity)

### Mass
We assign a `UniformMass` to our particles. In SOFA, mass is essential for dynamic simulations where forces result in acceleration ($F = ma$).

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Mass")
root.addObject("UniformMass", template="Vec1", name="mass", vertexMass=1.0)

### Fixed Constraint
To prevent the entire system from drifting or falling, we use a `FixedProjectiveConstraint`. Here, we fix the first particle (index 0) in space.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Constraint.Projective")
root.addObject("FixedProjectiveConstraint", template="Vec1", name="fixedConstraint", indices=[0])

### Spring Force Field
The `SpringForceField` manages elastic forces between particles. We define a `LinearSpring` connecting particle 0 and particle 1.

**Key Parameters**:
- `springStiffness`: The "hardness" of the spring.
- `dampingFactor`: The energy loss during motion (prevents infinite oscillation).
- `restLength`: The length at which the spring exerts no force.

In [ ]:
from Sofa.SofaDeformable import LinearSpring

# Define the spring properties
spring = LinearSpring(index1=0, index2=1, 
                      springStiffness=0.1, 
                      dampingFactor=0.1, 
                      restLength=0.5, 
                      elongationOnly=False)

SofaRuntime.importPlugin("Sofa.Component.SolidMechanics.Spring")
spring_force_field = root.addObject("SpringForceField", template="Vec1", name="springs")
spring_force_field.addSpring(spring)

## 4. Running the Simulation
We initialize the scene and execute the simulation loop. We will record the position of the second particle over time.

In [ ]:
Sofa.Simulation.initRoot(root)

dt = root.dt.value
current_time = 0.0
time_steps = []
positions = []

def retrieve_position():
    """Extracts the 1D position of the second particle."""
    current_pos = root.particles.position.value
    # Particle 1 is at index 1
    second_particle_pos = current_pos[1][0]
    time_steps.append(current_time)
    positions.append(second_particle_pos)

# Initial record
retrieve_position()

# Run the simulation for 10,000 steps
for iteration in range(10000):
    Sofa.Simulation.animate(root, dt)
    current_time += dt
    # Record data every step (or sampled)
    retrieve_position()

## 5. Visualization and Analysis
Finally, we use `bokeh` to plot the oscillation of the mass.

In [ ]:
from bokeh.plotting import figure, output_notebook, show
output_notebook() # Enable inline plotting for Jupyter Notebook

# Create a Bokeh plot
p = figure(title="Mass-Spring Oscillation (1D)", 
           x_axis_label="Time (s)", 
           y_axis_label="Position (m)", 
           width=800, height=400)

# Plot the position data
p.line(time_steps, positions, line_width=2, line_color="blue", legend_label="Particle 1 Position")

# Add the rest length horizontal line
from bokeh.models import Span
rest_length_line = Span(location=0.5, dimension='width', line_color='red', line_dash='dashed', line_width=2)
p.add_layout(rest_length_line)

# Add a dummy line for the legend entry of the rest length (Spans don't appear in legend automatically)
p.line([], [], line_color="red", line_dash="dashed", line_width=2, legend_label="Rest Length (0.5m)")

p.legend.location = "top_right"
p.grid.grid_line_alpha = 0.3

show(p)

### Summary
The resulting graph shows a **damped harmonic oscillation**. 
- The particle starts at position `1.0`.
- The spring's rest length is `0.5`, creating an initial displacement.
- The `dampingFactor` causes the oscillation amplitude to decrease over time until the particle eventually settles at the rest length.

## 6. Exercises
Now it's your turn! Experiment with different parameters to see how they affect the system.

### Exercise 1: Changing Stiffness
Modify the `springStiffness` parameter in the `LinearSpring` definition. What happens when you increase it to `0.5`? Observe the change in oscillation frequency and initial force.

### Exercise 2: Critical Damping
Adjust the `dampingFactor`. Can you find a value where the particle returns to the rest length as quickly as possible without overshooting (oscillating past it)? This is known as **critical damping**.

### Exercise 3: Change the Mass
Change the `vertexMass` in the `UniformMass` component. How does a larger mass (e.g., `5.0`) affect the frequency of the oscillation? How does it compare to the effect of stiffness?

### Exercise 4: Gravity and Spring
The gravity was initially set to zero (`root.gravity.value = [0, 0, 0]`). What happens if you re-enable it (e.g., `[-9.81, 0, 0]`)? Note that since the simulation is 1D (`Vec1`), only the first component of the gravity vector will be used. How does gravity change the equilibrium position (the position where the particle eventually stops)?